In [ ]:
from data.get_tickers import get_tickers
from data.get_stock_data import get_ohlcv_batch
from mean.base import MeanModelBatch 
from mean.local_linear_trend import LLTModel 
from mean.local_level import LLModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tickers = get_tickers().query("Market == 'Oslo Børs'").Symbol.to_list()
ohlcv_batch = get_ohlcv_batch(tickers, interval ="1h", update=False).drop_illiquid(1_000_000).drop_failed()

In [ ]:
llt_batch = MeanModelBatch.fit(batch=ohlcv_batch, model=LLTModel)
llt_batch.summary().sort_values("ret", ascending=False)

In [ ]:
llt_batch.models["FRO.OL"].plot(skip=1000)

In [ ]:
ll_batch = MeanModelBatch.fit(batch=ohlcv_batch, model=LLModel)
ll_batch.summary().sort_values("ret", ascending=False)

In [ ]:
ll_batch.models["AKRBP.OL"].plot(skip=1000)